In [1]:
from keys import * #api tokens, logins: CLIENT_ID, SECRET_KEY, USERNAME, PASSWORD, USER_AGENT

import praw
import pandas as pd

In [2]:
reddit = praw.Reddit(client_id = CLIENT_ID,
                     client_secret = SECRET_KEY,
                     user_agent = USER_AGENT,
                     username = USERNAME,
                     password = PASSWORD
                    ) 

In [3]:
subred = reddit.subreddit("hungary")

In [40]:
new = subred.new(limit = 100) #search for last 100 posts in "new" section

In [41]:


row_list = []
for post in new:

    try:
        #Post metadata and content
        post_id = post.id
        flair = post.link_flair_text
        author = post.author.name
        timestamp = post.created_utc #unix time, UTC, -2 hours earlier than us
        upvotes = post.score 
        text = post.selftext #it is empty if the text is a link #TODO test for pic and poll
        title = post.title 
        comments = post.comments
        awards = post.all_awardings

        #one post goes to one row in the df
        row = {
            "post_id" : post_id, #str
            "flair" : flair, #str
            "author" : author, #str
            "timestamp" : timestamp, #? str?
            "upvotes" : upvotes, #TODO datatypes
            "text" : text, #str
            "title" : title, #str
            "comments" : comments, # praw.CommentForest
            "awards": awards
        }

        row_list.append(row)

    except AttributeError:
        pass
post_data = pd.DataFrame(row_list)

In [46]:

row_list = []
for _,post_row in post_data.iterrows():

    for comment in post_row.comments:
        try:
            #Comment metadata and content
            comment_id = comment.id
            text = comment.body
            author = comment.author.name
            upvotes = comment.score
            timestamp = comment.created_utc

            #Post metadata
            flair = post_row.flair
            post_id = post_row.post_id
            title = post_row.title
            
            # one row goes to one row in the df
            row = {
                "comment_id" : comment_id, #str
                "text" : text, #str
                "author" : author, #str
                "upvotes" : upvotes, #? str?
                "timestamp" : timestamp, #
                "flair" : flair, #str
                "post_id" : post_id, #str
                "title" : title #str
            }
            row_list.append(row)
        except AttributeError:
            pass

comment_data = pd.DataFrame(row_list)

In [47]:
comment_data

,comment_id,text,author,upvotes,timestamp,flair,post_id,title
0,hfijnj8,https://nyugdijasotthon.hu/bekerulesi-idok,tt2019GA,1,1.633462e+09,DISCUSSION,q239hf,Idősotthon - tapasztalatok
1,hfigm34,"Az előválasztás eredményei előtt, annyira átha...",jsw_23,15,1.633460e+09,POLITICS,q22vfc,Márki-Zay Péter: Ha nem tudom meggyőzni Karács...
2,hfig4e7,ez már kezd komédia lenni,kozpenz,17,1.633460e+09,POLITICS,q22vfc,Márki-Zay Péter: Ha nem tudom meggyőzni Karács...
3,hfighpi,Dobrev < Karácsony\n\nOrbán < MZP\n\n A matek ...,Exodu199,13,1.633460e+09,POLITICS,q22vfc,Márki-Zay Péter: Ha nem tudom meggyőzni Karács...
4,hfikmi0,"Akkor a Gergely nyert! Jó éjszakát, szevasztok!",B4mbinoo,2,1.633462e+09,POLITICS,q22vfc,Márki-Zay Péter: Ha nem tudom meggyőzni Karács...
...,...,...,...,...,...,...,...,...
1232,hfd2yrg,Van baj a toronyban.,zsoltsandor,10,1.633363e+09,FOST,q184a2,A Nagyenyed utcai villamos megálló kincse
1233,hfd6yv9,nemzeti brüsszelség :-),megfog6od,7,1.633364e+09,FOST,q184a2,A Nagyenyed utcai villamos megálló kincse
1234,hfda883,Amugy ez a szoveg mit jelent? Tobbszori olvasa...,Freakonomistcat,7,1.633366e+09,FOST,q184a2,A Nagyenyed utcai villamos megálló kincse
1235,hfde6xn,Mire gondolt a költő,Valaki997,3,1.633367e+09,FOST,q184a2,A Nagyenyed utcai villamos megálló kincse
